# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [26]:
import pyorient

In [27]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [28]:
print(client.db_list())

{{'databases': {'gods': 'memory:gods'}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [29]:
# Creation d'une base de donnees
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

PyOrientDatabaseException: com.orientechnologies.orient.core.exception.ODatabaseException - Database named 'gods' already exists: gods

**Q:** Connect your pyorient client to the `gods` database.

In [30]:
# Ouverture de la base de donnees
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [31]:
# Creation d'un vertex
client.command("CREATE VERTEX V CONTENT {'name': 'Zeus', 'symbol': 'thunder'}")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [32]:
# Définition de la requête
request = client.command("SELECT * FROM V")

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB5C390>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB5CE80>, 'name': 'Zeus', 'symbol': 'thunder'}
{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB5CF98>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB5CD30>}
{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB1AD30>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB1ABA8>}
{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB1AB38>}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB1AA90>, 'name': 'Arès', 'symbol': 'weapons'}
{'name': 'Zeus', 'symbol': 'thunder'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [33]:
# Creation de plusieurs vertices
client.command("CREATE VERTEX V CONTENT {'name': 'Héra', 'symbol': 'tiara'}")
client.command("CREATE VERTEX V CONTENT {'name': 'Poséidon', 'symbol': 'trident'}")
client.command("CREATE VERTEX V CONTENT {'name': 'Athena', 'symbol': 'helmet'}")
client.command("CREATE VERTEX V CONTENT {'name': 'Arès', 'symbol': 'weapons'}")

**Q:** Display all vertices with name = `Arès`

In [34]:
# Définition de la requête
request = client.command("SELECT FROM V WHERE name = 'Arès'")

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'name': 'Arès', 'symbol': 'weapons'}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB13C88>, 'name': 'Arès', 'symbol': 'weapons'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [35]:
# Création d'une relation entre deux vertices
client.command("CREATE EDGE E FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Poséidon') CONTENT {'kind': 'sibling'}")

**Q:** Redisplay all vertices, discuss.

In [36]:
# Définition de la requête
request = client.command("SELECT * FROM V")

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB65748>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CAE5748>, 'name': 'Zeus', 'symbol': 'thunder'}
{'name': 'Athena', 'symbol': 'helmet'}
{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB304E0>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB305F8>}
{'name': 'Arès', 'symbol': 'weapons'}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB307F0>, 'name': 'Poséidon', 'symbol': 'trident', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30908>}
{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30AC8>}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30BA8>, 'name': 'Arès', 'symbol': 'weapons'}
{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30E10>}
{'name': 'Héra', 'symb

**Q:** Display all edges. They are contained in the class `E`

In [37]:
# Définition de la requête
request = client.command("SELECT * FROM E")

# Affichage du résultat
for edge in request:
    print(edge.oRecordData)

{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB13B00>, 'in': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB13F28>}
{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6C2B0>, 'in': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6C390>}
{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6C550>, 'in': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6C5F8>}
{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6C7B8>, 'in': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6C860>}
{'kind': 'father', 'out': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6CA20>, 'in': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6CAC8>}
{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6CC88>, 'in': <pyorient.otypes.OrientRecordLink object at 0x0000027A0CB6CD30>}
{'kind': 'father', 'out

Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [38]:
# Création de plusieurs relations
edges = [
    {'from': 'Zeus', 'to': 'Héra', 'kind': 'sibling'},
    {'from': 'Zeus', 'to': 'Arès', 'kind': 'father'},
    {'from': 'Zeus', 'to': 'Athena', 'kind': 'father'},
    {'from': 'Héra', 'to': 'Arès', 'kind': 'mother'},
    {'from': 'Héra', 'to': 'Zeus', 'kind': 'sibling'},
    {'from': 'Poséidon', 'to': 'Zeus', 'kind': 'sibling'}
]

# Application des relations
for edge in edges:
    client.command(f"CREATE EDGE E FROM (SELECT FROM V WHERE name = '{edge['from']}') TO (SELECT FROM V WHERE name = '{edge['to']}') CONTENT {{'kind': '{edge['kind']}'}}")

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [39]:
# Définition de la requête
request = client.command("SELECT EXPAND(out()) FROM V WHERE name = 'Zeus'")

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB307B8>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30048>, 'name': 'Poséidon', 'symbol': 'trident'}
{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30DA0>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30E48>, 'name': 'Héra', 'symbol': 'tiara'}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30CF8>, 'name': 'Arès', 'symbol': 'weapons'}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB30E80>, 'name': 'Athena', 'symbol': 'helmet'}
{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB306A0>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB35AC8>}
{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB35C18>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB35C88>, 'name': 'Poséidon', 'symbol': 'trident'}
{'out_': <pyorien

**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [40]:
# Définition de la requête
query = "SELECT FROM V WHERE @rid IN (SELECT in FROM E WHERE kind = 'father')"
request = client.command(query)

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB35390>}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB35780>, 'name': 'Arès', 'symbol': 'weapons'}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB355F8>, 'name': 'Athena', 'symbol': 'helmet'}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB35908>, 'name': 'Arès', 'symbol': 'weapons'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [41]:
# Définition de la requête
query = "SELECT FROM V WHERE @rid IN (SELECT out FROM E WHERE kind = 'mother')"
request = client.command(query)

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB653C8>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB65D68>, 'name': 'Héra', 'symbol': 'tiara'}
{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB54048>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000027A0CB540B8>}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [45]:
# Définition de la requête
query = "SELECT FROM V WHERE @rid IN (SELECT in FROM E WHERE out = (SELECT FROM V WHERE name = 'Zeus') AND kind = 'sibling')"
request = client.command(query)

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

PyOrientSQLParsingException: com.orientechnologies.orient.core.sql.OCommandSQLParsingException - Error parsing query:
SELECT FROM V WHERE @rid IN SELECT in FROM E WHERE out = (SELECT FROM V WHERE name = 'Zeus') AND kind = 'sibling'
                       ^
Encountered "" at line 1, column 21.
Was expecting one of:
    
	DB name="gods"

## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

**Q:** Create the following people: `peter`, `meredith`.

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


**Q:** Who bought Bolognese sauce?

**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.